<a href="https://colab.research.google.com/github/Brvilardi/BancoDeDados-P2-Neo4j-Spotify/blob/main/P2_Sportify_Neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa P2 - LinkedIn usando Neo4j


| Nome    | RA |
| -------- | ------- |
| Bruno Vilardi Bueno  | 19.00331-5     |
| Martin Ropke | 19.01592-5      |






## Setup do ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
time: 2.25 s (started: 2023-06-10 11:15:25 +00:00)


In [ ]:
!pip install ipython-autotime
!pip install neo4j
!pip install yfiles_jupyter_graphs neo4j graphdatascience
%load_ext autotime
from google.colab import output
output.enable_custom_widget_manager()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 14.9 s (started: 2023-06-10 11:15:27 +00:00)


## Exercício 1 – Escolher um dataset no Kaggle (Spotify)

tracks_features -> https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs

spotify_artist_data -> https://www.kaggle.com/datasets/adnananam/spotify-artist-stats

### Import e tratamento das bases de dados do Kaggle sobre Spotify

In [44]:
# Importa musicas
import pandas as pd
import ast


tracks = pd.read_csv("/content/drive/MyDrive/neo4j-p2/tracks_features.csv")
tracks['artists'] = tracks['artists'].apply(lambda x: ast.literal_eval(x)) # converte a string para lista
tracks['artists'] = tracks['artists'].apply(lambda x: [i.replace('"', "") for i in x]) # remove " do nome do artista
tracks = tracks.drop_duplicates(subset=['name', 'album', 'speechiness', 'track_number']) # remove duplicadas

tracks.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,[Rage Against The Machine],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,[Rage Against The Machine],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,[Rage Against The Machine],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,[Rage Against The Machine],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,[Rage Against The Machine],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


time: 29 s (started: 2023-06-10 11:34:08 +00:00)


In [ ]:
# Importa artistas

artists = pd.read_csv("/content/drive/MyDrive/neo4j-p2/spotify_artist_data.csv")
artists['Artist Name'] = artists['Artist Name'].replace('"','') # remove " do nome do artista
artists.head()

,Unnamed: 0,Artist Name,Lead Streams,Feats,Tracks,One Billion,100 Million,Last Updated
0,1,Drake,"50,162,292,808","19,246,513,666",262,6,130,19.09.22
1,2,Bad Bunny,"44,369,032,140","5,391,990,975",163,5,118,20.09.22
2,3,Ed Sheeran,"38,153,682,361","2,791,278,201",240,10,62,19.09.22
3,4,The Weeknd,"34,767,779,741","4,288,903,657",186,8,72,19.09.22
4,5,Taylor Swift,"32,596,728,109","424,053,296",323,1,96,20.09.22


time: 562 ms (started: 2023-06-10 11:16:16 +00:00)


In [ ]:
# Selecionando parte do dataset

# primeiros 8 artistas
artists_10 = artists[0:8] 

# todas as musicas que tem pelo menos um dos artistas envolvidos
tracks_10 = tracks[tracks['artists'].apply(lambda x: any(artist in x for artist in artists_10['Artist Name'].unique()))]

# atualizando a lista de artistas para conter todos os artistas envolvidos nas musicas acima
artists_10 = artists[artists['Artist Name'].isin(tracks_10.artists.explode().unique())] 


time: 56.9 s (started: 2023-06-10 11:16:17 +00:00)


In [ ]:
tracks_10.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
19254,1B87ZDrD3Pno9hS4stamWO,Comin Out Strong (feat. The Weeknd),HNDRXX,7K6OykPbezfgKgBufihn6X,"[Future, The Weeknd]","['1RyvyyTE3xzB2ZywiAwp0i', '1Xyo4u8uXC1ZmMpatF...",2,1,True,0.628,...,0.4750,0.290,0.0,0.1180,0.330,108.761,254370,5.0,2017,2017-07-27
19325,70eFcWOvlMObDhURTqT4Fv,Beautiful People (feat. Khalid),No.6 Collaborations Project,3oIFxDIo2fwuk4lwCmFZCx,"[Ed Sheeran, Khalid]","['6eUKZXaKkcviH0Ku9w2n3V', '6LuN9FCkKOj5Pcnpou...",1,1,False,0.640,...,0.1860,0.124,0.0,0.0801,0.546,92.979,197867,4.0,2019,2019-07-12
19326,4vUmTMuQqjdnvlZmAH61Qk,South of the Border (feat. Camila Cabello & Ca...,No.6 Collaborations Project,3oIFxDIo2fwuk4lwCmFZCx,"[Ed Sheeran, Camila Cabello, Cardi B]","['6eUKZXaKkcviH0Ku9w2n3V', '4nDoRrQiYLoBzwC5Bh...",2,1,False,0.857,...,0.0825,0.148,0.0,0.0865,0.668,97.989,204467,4.0,2019,2019-07-12
19327,4wuCQX7JvAZLlrcmH4AeZF,Cross Me (feat. Chance the Rapper & PnB Rock),No.6 Collaborations Project,3oIFxDIo2fwuk4lwCmFZCx,"[Ed Sheeran, Chance the Rapper, PnB Rock]","['6eUKZXaKkcviH0Ku9w2n3V', '1anyVhU62p31KFi8ME...",3,1,True,0.746,...,0.1200,0.212,0.0,0.0669,0.607,95.005,206187,4.0,2019,2019-07-12
19328,1AI7UPw3fgwAFkvAlZWhE0,Take Me Back to London (feat. Stormzy),No.6 Collaborations Project,3oIFxDIo2fwuk4lwCmFZCx,"[Ed Sheeran, Stormzy]","['6eUKZXaKkcviH0Ku9w2n3V', '2SrSdSvpminqmStGEL...",4,1,False,0.885,...,0.2160,0.219,0.0,0.1620,0.605,138.058,189733,4.0,2019,2019-07-12


time: 31.5 ms (started: 2023-06-10 11:17:14 +00:00)


In [ ]:
artists_10.head()

,Unnamed: 0,Artist Name,Lead Streams,Feats,Tracks,One Billion,100 Million,Last Updated
0,1,Drake,"50,162,292,808","19,246,513,666",262,6,130,19.09.22
1,2,Bad Bunny,"44,369,032,140","5,391,990,975",163,5,118,20.09.22
2,3,Ed Sheeran,"38,153,682,361","2,791,278,201",240,10,62,19.09.22
3,4,The Weeknd,"34,767,779,741","4,288,903,657",186,8,72,19.09.22
4,5,Taylor Swift,"32,596,728,109","424,053,296",323,1,96,20.09.22


time: 12.9 ms (started: 2023-06-10 11:17:14 +00:00)


## Setup Neo4j

In [ ]:
# Conecta com o banco
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://393f3617.databases.neo4j.io"
AUTH = ("neo4j", "it-AfG0hmi3F26d4hQF8SIFeCdFGhQtjLL1RFArfmbw")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

driver = GraphDatabase.driver(URI, auth=AUTH)


time: 2.02 s (started: 2023-06-10 11:17:14 +00:00)


In [ ]:
#limpa o banco
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")

time: 2.28 s (started: 2023-06-10 11:17:16 +00:00)


## Exercício2 – Importe os dados para o Neo4J 

#### Import dos Artistas

In [ ]:
# Importanto artistas
with driver.session() as session:
    for _, row in artists_10.iterrows():
        session.run("CREATE (:Artist {name: $name, lead_streams: $lead_streams, feats: $feats})", 
                    name=row["Artist Name"],
                    lead_streams=row["Lead Streams"],
                    feats=row["Feats"])


time: 17.8 s (started: 2023-06-10 11:17:18 +00:00)


### Import das Músicas

In [ ]:
# Importanto Tracks

with driver.session() as session:
    for _, row in tracks_10.iterrows():
        session.run("CREATE (:Track {name: $name, album: $album, year: $year, danceability: $danceability, explicit: $explicit, energy: $energy, duration_ms: $duration_ms})", 
                    name=row["name"],
                    album=row["album"],
                    year=row["year"],
                    danceability=row["danceability"],
                    explicit=row["explicit"],
                    energy=row["energy"],
                    duration_ms=row["duration_ms"],
                    )


time: 45 s (started: 2023-06-10 11:17:36 +00:00)


### Criar relacionamentos

In [ ]:
# Fazendo o match entre artistas e musicas

for _, track in tracks_10.iterrows():
  track_name = track['name']
  for artist_name in track['artists']:
    with driver.session() as session:       
        print(f"{artist_name} - [:PERFORMS] -> {track_name}")

        res = session.run(f"""MATCH (a:Artist), (t:Track )
        WHERE a.name= "{artist_name}" AND t.name = "{track_name}"
        MERGE (a)-[:PERFORMS]->(t)
        """)
        
    
  



Future - [:PERFORMS] -> Comin Out Strong (feat. The Weeknd)
The Weeknd - [:PERFORMS] -> Comin Out Strong (feat. The Weeknd)
Ed Sheeran - [:PERFORMS] -> Beautiful People (feat. Khalid)
Khalid - [:PERFORMS] -> Beautiful People (feat. Khalid)
Ed Sheeran - [:PERFORMS] -> South of the Border (feat. Camila Cabello & Cardi B)
Camila Cabello - [:PERFORMS] -> South of the Border (feat. Camila Cabello & Cardi B)
Cardi B - [:PERFORMS] -> South of the Border (feat. Camila Cabello & Cardi B)
Ed Sheeran - [:PERFORMS] -> Cross Me (feat. Chance the Rapper & PnB Rock)
Chance the Rapper - [:PERFORMS] -> Cross Me (feat. Chance the Rapper & PnB Rock)
PnB Rock - [:PERFORMS] -> Cross Me (feat. Chance the Rapper & PnB Rock)
Ed Sheeran - [:PERFORMS] -> Take Me Back to London (feat. Stormzy)
Stormzy - [:PERFORMS] -> Take Me Back to London (feat. Stormzy)
Ed Sheeran - [:PERFORMS] -> Best Part of Me (feat. YEBBA)
Yebba - [:PERFORMS] -> Best Part of Me (feat. YEBBA)
Ed Sheeran - [:PERFORMS] -> I Don't Care (with 

### Visualizar os grafos

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from pprint import pprint

def visualize_neo4j_graph(driver, cypher_query, graph_layout="organic"):
    """
    Visualize neo4j graph, expect queries with two nodes (n / m) and relationship r
    """
    styles = {
    "Artist": {"color":"#FF8C00", "shape":"ellipse", "label":"name"},
    "Track": {"color":"#40E0D0", "shape":"ellipse", "label":"label"}
    }

    # Execute the Cypher query and retrieve the graph data
    with driver.session() as session:
        result = session.run(cypher_query)
        graph = result.graph()
        # pprint(result.data())
    w = GraphWidget(graph = graph)
    w.set_node_styles_mapping(lambda index, node : styles.get(node["properties"]["label"], {}))
    w.set_node_label_mapping(lambda index, node : node["properties"][styles.get(node["properties"]["label"], {"label":"label"})["label"]])
    w.graph_layout = graph_layout
    w.show()
    

time: 94 ms (started: 2023-06-10 11:22:17 +00:00)


#### Grafo de um Artista

In [ ]:
visualize_neo4j_graph(driver, "MATCH (a:Artist)-[r:PERFORMS]->(t:Track) WHERE a.name = 'Ed Sheeran' RETURN * LIMIT 50", 
                      graph_layout="circular")

GraphWidget(layout=Layout(height='500px', width='100%'))

time: 820 ms (started: 2023-06-10 11:22:17 +00:00)


#### Grafo de uma Música

In [ ]:
visualize_neo4j_graph(driver, "MATCH (a:Artist)-[r:PERFORMS]->(t:Track) WHERE t.name = 'Mood (Remix) feat. Justin Bieber, J Balvin & iann dior' RETURN * LIMIT 20")

GraphWidget(layout=Layout(height='500px', width='100%'))

time: 383 ms (started: 2023-06-10 11:22:18 +00:00)


#### Grafo completo

In [ ]:
visualize_neo4j_graph(driver, "MATCH (a:Artist)-[r:PERFORMS]->(t:Track) RETURN *")

GraphWidget(layout=Layout(height='500px', width='100%'))

time: 1.43 s (started: 2023-06-10 11:22:18 +00:00)


## EX 4 - A rede de relacionamentos no nível mais próximo e no segundo nível.

#### Nível mais próximo

In [ ]:
from pprint import pprint
#a rede de relacionamentos no nível mais próximo

ARTIST_NAME = "Ed Sheeran"

with driver.session() as session:
      res = session.run("""
      MATCH (a:Artist)-[:PERFORMS]->(:Track)<-[:PERFORMS]-(relatedArtist:Artist)
      WHERE a.name = $name
      RETURN DISTINCT relatedArtist.name AS name
      ORDER BY relatedArtist.name 
      """, 
      name=ARTIST_NAME)
      nomes = [i['name'] for i in res.data()]

print(f"Nomes dos artistas de nível mais próximo a {ARTIST_NAME}")
pprint(nomes)


Nomes dos artistas de nível mais próximo a Ed Sheeran
['50 Cent',
 'A Boogie Wit da Hoodie',
 'Beyoncé',
 'Bruno Mars',
 'Burna Boy',
 'Camila Cabello',
 'Cardi B',
 'Chance the Rapper',
 'Cheat Codes',
 'Chris Stapleton',
 'Dave',
 'Ella Mai',
 'Eminem',
 'Galantis',
 'H.E.R.',
 'Joel Corry',
 'Jonas Blue',
 'Justin Bieber',
 'Khalid',
 'Loud Luxury',
 'Meek Mill',
 'Paulo Londra',
 'PnB Rock',
 'Sam Feldt',
 'Skrillex',
 'Stormzy',
 'Travis Scott',
 'Young Thug']
time: 415 ms (started: 2023-06-10 11:22:20 +00:00)


In [ ]:
# Visualização dos nodes
visualize_neo4j_graph(driver, f"MATCH (a:Artist)-[r1:PERFORMS]->(t:Track)<-[r2:PERFORMS]-(relatedArtist:Artist) WHERE relatedArtist.name = '{ARTIST_NAME}' RETURN *",
                      graph_layout='radial')

GraphWidget(layout=Layout(height='500px', width='100%'))

time: 667 ms (started: 2023-06-10 11:22:20 +00:00)


#### Segundo nível

In [ ]:
from pprint import pprint
#a rede de relacionamentos no nível mais próximo e no segundo nível.

ARTIST_NAME = "Ed Sheeran"

with driver.session() as session:
      res = session.run("""MATCH (a1:Artist)-[:PERFORMS]->(t1:Track)<-[:PERFORMS]-(a2:Artist)-[:PERFORMS]->(t2:Track)<-[:PERFORMS]-(a3:Artist)
      WHERE a1 <> a2 AND a2 <> a3 AND a1 <> a3
      AND a1.name = $name
      AND NOT (a1)-[:PERFORMS]->(:Track)<-[:PERFORMS]-(a3)
      RETURN DISTINCT a3.name AS name
      ORDER BY name
      """, 
      name=ARTIST_NAME)
      nomes = [i['name'] for i in res.data()]

print(f"Nomes dos artistas de nível mais próximo a {ARTIST_NAME}")
pprint(nomes)


Nomes dos artistas de nível mais próximo a Ed Sheeran
['24kGoldn',
 'Ariana Grande',
 'Avril Lavigne',
 'Bad Bunny',
 'Bryan Adams',
 'Calvin Harris',
 'Chris Brown',
 'DJ Khaled',
 'Dan + Shay',
 'David Guetta',
 'Diplo',
 'Drake',
 'Gucci Mane',
 'J Balvin',
 'Jack Ü',
 'Lil Wayne',
 'Major Lazer',
 'Marshmello',
 'Michael Bublé',
 'MØ',
 'P!nk',
 'Pharrell Williams',
 'Quavo',
 'SZA',
 'The Notorious B.I.G.',
 'The Weeknd',
 'Trey Songz',
 'iann dior']
time: 711 ms (started: 2023-06-10 11:22:21 +00:00)


In [ ]:
# Visualização dos nodes

ARTIST_NAME = "Ed Sheeran"


visualize_neo4j_graph(driver, 
      f"""MATCH (a1:Artist)-[r1:PERFORMS]->(t1:Track)<-[r2:PERFORMS]-(a2:Artist)-[r3:PERFORMS]->(t2:Track)<-[r4:PERFORMS]-(a3:Artist)
      WHERE a1 <> a2 AND a2 <> a3 AND a1 <> a3
      AND a1.name = '{ARTIST_NAME}'
      AND NOT (a1)-[:PERFORMS]->(:Track)<-[:PERFORMS]-(a3)
      RETURN *""",
                      graph_layout='radial')

GraphWidget(layout=Layout(height='500px', width='100%'))

time: 1.26 s (started: 2023-06-10 11:22:22 +00:00)


### A força do relacionamento para chegar a um determinado nó

In [41]:

ARTISTA_1 = 'Ed Sheeran'
ARTISTA_2 = "Childish Gambino"

with driver.session() as session:
      res = session.run("""MATCH (a1:Artist { name:$name1 }),(a2:Artist { name:$name2 }), 
      p = shortestPath((a1)-[*..15]-(a2))
      RETURN length(p) as distancia
      """, 
      name1=ARTISTA_1,
      name2=ARTISTA_2)
      resp = res.data()[0]['distancia'] 

print(f"Distancia entre os artistas {ARTISTA_1} e {ARTISTA_2} é de {int(resp/2-1)}")





Distancia entre os artistas Ed Sheeran e Childish Gambino é de 2
time: 372 ms (started: 2023-06-10 11:32:43 +00:00)


In [42]:
# Visualização dos nodes


visualize_neo4j_graph(driver, 
      f"""MATCH (a1:Artist {{ name:'{ARTISTA_1}' }}),(a2:Artist {{ name:'{ARTISTA_2}' }}),
      p = shortestPath((a1)-[*..15]-(a2))
      RETURN *
      """,
                      graph_layout='radial')

GraphWidget(layout=Layout(height='500px', width='100%'))

time: 408 ms (started: 2023-06-10 11:32:46 +00:00)
